# Problèmes

### 1)
Primal: $\underset{x \in \mathbb{R}^3}{\min} \frac{1}{2} x' Q x + c' x ~~~~~~~~ s.t. ~ A x = b ~, ~~ x \ge 0$

with  $Q = \begin{bmatrix} 6 & 2 & 1\\ 2 & 5 & 2\\ 1 & 2 & 4 \end{bmatrix} $, $c = \begin{bmatrix} -8 \\ -3 \\ -3 \end{bmatrix}$, $A = \begin{bmatrix} 1 & 0 &1 \\ 0 & 1 & 1 \end{bmatrix}$ , and $b = \begin{bmatrix} 3 \\ 0 \end{bmatrix} $.


### 2) 
AFIRO        28 rows    32 cols        88 nonzeros       794 bytes      -4.6475314286E+02 optimalvalue

In [1]:
using LinearAlgebra
using LaTeXStrings
using DataFrames
using DataStructures
using QuadraticModels
using Printf
using SparseArrays
using BenchmarkTools
using NLPModels
using LinearOperators
using QPSReader
using SolverTools
using SolverBenchmark

# Mehrotra formulation plus générale:

min $c_0 + c'x + \frac{1}{2} x'Hx ~~~~$ s.t. $ Ax = b $ and $l \le x \le u$ 

In [144]:
function display_results(result)
    # fonction pour l'affichage
    println("\n-----------------------------------------------------------------------")
    println("------------------------------- RESULTS -------------------------------")
    result
end

function init_x0(lvar, uvar)
    # choice of an init point x0
    x0 = zeros(length(lvar))
    for i=1:length(x0)
        if lvar[i] == -Inf && uvar[i] == Inf
            x0[i] = 0.
        elseif lvar[i] == -Inf && uvar[i] != Inf
            x0[i] = uvar[i] - 1.
        elseif lvar[i] != -Inf && uvar[i] == Inf
            x0[i] = lvar[i] + 1.
        else
            x0[i] = (lvar[i] + uvar[i]) / 2 
        end
    end  
    return x0
end

function init_x0_lsq(A, b, lvar, uvar)
    x_tilde = A\b
    n = length(x_tilde)
    for i=1:n
        if x_tilde[i] <= lvar[i]
            x_tilde[i] = lvar[i] + 1.
        elseif uvar[i] <= x_tilde[i]
            x_tilde[i] = uvar[i] - 1.
        end
        if !(lvar[i] < x_tilde[i] < uvar[i])
            x_tilde[i] = (lvar[i] + uvar[i]) / 2.
        end
    end

    return x_tilde
end

function starting_points(Q, A, b, c, lvar, uvar, ilow, iupp, irng, J_augm, ρ, n_rows, n_cols)
    
    tmp_diag = vcat(-ρ*ones(n_cols), spzeros(n_rows))
    
    J_fact = ldlt(Symmetric(J_augm-Diagonal(tmp_diag)))
    init_xλ = J_fact \ [c ; b ]
    x0 = init_xλ[1:n_cols]
    λ0 = init_xλ[n_cols+1:end]
    s0_l, s0_u = zeros(n_cols), zeros(n_cols)
    dual_val = Q*x0 - A'*λ0 + c
    s0_l[ilow] = dual_val[ilow]
    s0_u[iupp] = -dual_val[iupp]
    
    x0_l1, x0_u1 = x0[ilow], x0[iupp]
    if length(ilow) == 0
        δx_l1, δs_l1 = 0., 0.
    else
        δx_l1 = max(-1.5*minimum(x0_l1 - lvar[ilow]), 0.)
        δs_l1 = max(-1.5*minimum(s0_l[ilow]), 0.)
    end
    
    if length(iupp) == 0
        δx_u1, δs_u1 = 0., 0.
    else
        δx_u1 = max(-1.5*minimum((uvar[iupp] - x0_u1)), 0.)
        δs_u1 = max(-1.5*minimum(s0_u[iupp]), 0.)
    end
    
    x0_l1 .+= δx_l1
    x0_u1 .-= δx_u1
    s0_l1 = s0_l[ilow] .+ δs_l1
    s0_u1 = s0_u[iupp] .+ δs_u1
    xs_l1, xs_u1 = s0_l1'*(x0_l1 - lvar[ilow]), s0_u1'*(uvar[iupp] - x0_u1)
    δx_l2 = δx_l1 + 0.5 *  xs_l1 / sum(s0_l1)
    δx_u2 = δx_u1 + 0.5 *  xs_u1 / sum(s0_u1)
    δs_l2 = δs_l1 + 0.5 * xs_l1 / sum(x0_l1-lvar[ilow])
    δs_u2 = δs_u1 + 0.5 * xs_u1 / sum(uvar[iupp]-x0_u1)
  
    x0[ilow] .+= δx_l2 
    x0[iupp] .-= δx_u2
    s0_l[ilow] = s0_l[ilow] .+ δs_l2
    s0_u[iupp] = s0_u[iupp] .+ δs_u2
    
    for i in irng
        if (lvar[i] < x0[i] < uvar[i]) == false
            x0[i] = (lvar[i] + uvar[i]) / 2.
        end
    end

    @assert all(x0 .> lvar) && all(x0 .< uvar)
    @assert all(s0_l[ilow] .> 0) && all(s0_u[iupp] .> 0)
    
    return x0, λ0, s0_l, s0_u
end
    
    

starting_points (generic function with 2 methods)

In [145]:
function compute_α_dual(v, dir_v)
    n = length(v)
    if n == 0
        return 1.
    end
    α = 1.
    for i=1:n
        if dir_v[i] < 0.
            α_new = -v[i] * 0.999 / dir_v[i]
            if α_new < α
                α = α_new
            end
        end
    end
    return α
end


    
function compute_α_primal(v, dir_v, lvar, uvar)
    n = length(v)
    α_l, α_u = 1., 1.
    for i=1:n
        if dir_v[i] > 0.
            α_u_new = (uvar[i] - v[i]) * 0.999 / dir_v[i]
            if α_u_new < α_u
                α_u = α_u_new
            end
        elseif dir_v[i] < 0.
            α_l_new = (lvar[i] - v[i]) * 0.999 / dir_v[i]
            if α_l_new < α_l
                α_l = α_l_new
            end
        end
    end
    return min(α_l, α_u)
end

function compute_μ(x_l, x_u, s_l, s_u, lvar, uvar, nb_low, nb_upp)
    #x_l coordinates of x corresponding to finite lower bounds ( resp. finite upper bounds for x_u)
    # arguments must have finite bounds 
    return (s_l' * (x_l-lvar) + s_u' * (uvar-x_u)) / (nb_low + nb_upp)
end


function is_in_Neighborhood_inf(gamma, x_l, x_u, s_l, s_u, lvar, uvar)
    # check if the current point is in N_inf(gamma)
    # true : (xi_l - lvari) * si_l >= gamma mu   and   (uvari - xi_u) * si_u >= gamma mu 
    mu = Compute_mu(x_l, x_u, s_l, s_u, lvar, uvar)
    for i=1:length(x_l)
        if (x_l[i] - lvar[i]) * s_l[i] < gamma*mu
            return false
        end
    end
    for i=1:length(x_u)
        if (uvar[i] - x_u[i]) * s_u[i] < gamma*mu
            return false
        end
    end
    return true
end

is_in_Neighborhood_inf (generic function with 1 method)

In [146]:
function solve_augmented_system_aff(J_fact, rc, rb, x, s_l, s_u, ilow, 
                                    iupp, lvar, uvar, n_cols)
    
    F_x_λ_aff = [-rc 
                 -rb]
    F_x_λ_aff[ilow] += s_l[ilow]
    F_x_λ_aff[iupp] -= s_u[iupp]
    
    Δ_x_λ_aff = J_fact\F_x_λ_aff
    Δ_aff = [Δ_x_λ_aff
             -s_l[ilow] - s_l[ilow].*Δ_x_λ_aff[1:n_cols][ilow]./(x[ilow]-lvar[ilow])
             - s_u[iupp] - s_u[iupp].*Δ_x_λ_aff[1:n_cols][iupp]./(x[iupp]-uvar[iupp])]
    return Δ_aff
end

function solve_augmented_system_cc(J_fact, σ, μ,x, s_l, s_u, ilow, iupp,  lvar, uvar,
                                   Δ_aff, n_cols, n_rows, n_low; sparse=true)
    
    
    rxs_l = -σ*μ .+ Δ_aff[1:n_cols][ilow].*Δ_aff[n_rows+n_cols+1: n_rows+n_cols+n_low]
    rxs_u = σ*μ .+ Δ_aff[1:n_cols][iupp].*Δ_aff[n_rows+n_cols+n_low+1: end]
    
    if sparse
        F_x_λ_cc = spzeros(n_cols+n_rows,1)
    else
        F_x_λ_cc = zeros(n_cols+n_rows,1)
    end
    
    F_x_λ_cc[ilow] += rxs_l./(x[ilow]-lvar[ilow])
    F_x_λ_cc[iupp] -= rxs_u./(x[iupp]-uvar[iupp])
    
#     F_x_λ_cc = [ I_low*inv_X_L*rxs_l + I_op_upp*inv_X_U*rxs_u
#                                    Or                 ]
    Δ_x_λ_cc = J_fact\F_x_λ_cc
    Δ_cc = [Δ_x_λ_cc
            -(rxs_l+s_l[ilow].*Δ_x_λ_cc[1:n_cols][ilow])./(x[ilow]-lvar[ilow])
            -(rxs_u+s_u[iupp].*Δ_x_λ_cc[1:n_cols][iupp])./(x[iupp]-uvar[iupp]) ]
    return Δ_cc
end


solve_augmented_system_cc (generic function with 1 method)

In [193]:
function mehrotraPCQuadBounds(QM; max_iter=300, ϵ_pdd=1e-8, ϵ_rb=1e-6, ϵ_rc=1e-6,
                              tol_Δx=1e-16, ϵ_μ=1e-16, display=true)
    
    start_time = time()
    elapsed_time = 0.0
    
    # get variables from QuadraticModel
    lvar, uvar = QM.meta.lvar, QM.meta.uvar
    n_cols = length(lvar)
    Oc = zeros(n_cols)
    ilow, iupp = [QM.meta.ilow; QM.meta.irng], [QM.meta.iupp; QM.meta.irng] # finite bounds index
    n_low, n_upp = length(ilow), length(iupp) # number of finite constraints
    A = jac(QM, Oc)
    n_rows, n_cols = size(A) 
    Q = hess(QM, Oc)
    c = grad(QM, Oc)
    c0 = obj(QM, Oc)
    @assert QM.meta.lcon == QM.meta.ucon # equality constraint (Ax=b)
    b = QM.meta.lcon
    
    # init regularization values
    ρ, δ = 1.e-8, 1.e-6
    J_augm = [-Q       A'
                A     δ*I ]
    
    
    x, λ, s_l, s_u = starting_points(Q, A, b, c, lvar, uvar, ilow, iupp, 
                                     QM.meta.irng, J_augm, 1e-6, n_rows, n_cols)


    
#     x = init_x0_lsq(A, b, lvar, uvar)
#     @assert all(x .> lvar) && all(x .< uvar)
#     s_l, s_u = copy(Oc), copy(Oc)
#     s_l[ilow] .= 1.
#     s_u[iupp] .= 1.
    Qx = Q * x
#     λ = sparse(A') \ (c+Qx) # least square initialisation, s_0 = stilde_0
    
    rb = A * x - b
    rc = -Qx + A' * λ + s_l - s_u - c
    μ = compute_μ(x[ilow], x[iupp], 
                  s_l[ilow], s_u[iupp], 
                  lvar[ilow], uvar[iupp],
                  n_low, n_upp)

    k = 0

    # stopping criterion
    #n_c_p1 =  norm(c, Inf) + 1.
    #n_b_p1 =  norm(b, Inf) + 1.
    xTQx_2 = x' * Qx / 2
    cTx = c' * x
    pri_obj = xTQx_2 + cTx + c0
    dual_obj = b' * λ - xTQx_2 + s_l[ilow]'*lvar[ilow] - s_u[iupp]'*uvar[iupp] +c0
    pdd = abs(pri_obj - dual_obj ) / (1 + abs(pri_obj)) 
    max_rc, max_rb = norm(rc, Inf), norm(rb, Inf)
    optimal = pdd < ϵ_pdd && max_rc < ϵ_rc && max_rb < ϵ_rb
    n_Δx = 0.
    small_Δx, small_μ = false, μ < ϵ_μ
    

    # display
    if display == true
        @info log_header([:k, :pri_obj, :pdd, :max_rb, :max_rc, :n_Δx, :μ],
                         [Int, Float64, Float64, Float64, Float64, Float64, Float64, Float64],
                        hdr_override=Dict(:k=>"Iter", :pri_obj=>"primal", :pdd=>"pdd",
                                          :max_rb=>"rb cond", :max_rc=>"rc cond",
                                          :n_Δx=>"‖Δx‖", :μ=>"μ"))
        @info log_row(Any[k, pri_obj, pdd, max_rb, max_rc, n_Δx, μ])
    end

    while k<max_iter && !optimal && !small_Δx && !small_μ
        
            
            # Affine scaling direction
        
#         J_augm = [-Q-I_low*inv_X_L*S_low-I_op_upp*inv_X_U*S_upp-ρ*I        A'
#                                A                                         ρ*I  ]
        
        #J_augm[1:n_cols, 1:n_cols] = -Q - ρ*I
        #J_augm[1:n_cols+1:n_cols^2] = -Q[1:n_cols+1:n_cols^2] .- ρ
        J_augm[diagind(J_augm)[1:n_cols]] = -Q[diagind(Q)] .- ρ
        for j in ilow
            J_augm[j,j] -=  s_l[j] / (x[j] - lvar[j]) 
        end
        for j in iupp
            J_augm[j,j] += s_u[j] / (x[j] - uvar[j])
        end

        J_fact = ldlt(Symmetric(J_augm))
        #J_fact = lu(J_augm, check=true)
        
        Δ_aff = solve_augmented_system_aff(J_fact, rc, rb, x, s_l, s_u, ilow, 
                                           iupp, lvar, uvar, n_cols)

        α_aff_pri = compute_α_primal(x, Δ_aff[1:n_cols], lvar, uvar)
        α_aff_dual_l = compute_α_dual(s_l[ilow], Δ_aff[n_rows+n_cols+1: n_rows+n_cols+n_low])
        α_aff_dual_u = compute_α_dual(s_u[iupp], Δ_aff[n_rows+n_cols+n_low+1:end])

        # alpha_aff_dual_final is the min of the 2 alpha_aff_dual
        α_aff_dual_final = min(α_aff_dual_l, α_aff_dual_u)
        
        μ_aff = compute_μ(x[ilow] + α_aff_pri * Δ_aff[1:n_cols][ilow],
                          x[iupp] + α_aff_pri * Δ_aff[1:n_cols][iupp],
                          s_l[ilow] + α_aff_dual_final * Δ_aff[n_rows+n_cols+1: n_rows+n_cols+n_low],
                          s_u[iupp] + α_aff_dual_final * Δ_aff[n_rows+n_cols+n_low+1: end],
                          lvar[ilow], uvar[iupp],
                          n_low, n_upp)
        
        σ = (μ_aff / μ)^3

        # corrector and centering step
        
        Δ_cc = solve_augmented_system_cc(J_fact, σ, μ,x, s_l, s_u, ilow, iupp,  lvar, uvar,
                                         Δ_aff, n_cols, n_rows, n_low, sparse=true)
                ### sparse = false when using lu
        
        Δ = Δ_aff + Δ_cc # final direction
        
        α_pri = compute_α_primal(x, Δ[1:n_cols], lvar, uvar)
        α_dual_l = compute_α_dual(s_l[ilow], Δ[n_rows+n_cols+1: n_rows+n_cols+n_low])
        α_dual_u = compute_α_dual(s_u[iupp], Δ[n_rows+n_cols+n_low+1: end])

        α_dual_final = min(α_dual_l, α_dual_u)

        # new parameters
        x += α_pri * Δ[1:n_cols]
        λ += α_dual_final * Δ[n_cols+1: n_rows+n_cols]
        s_l[ilow] += α_dual_final * Δ[n_rows+n_cols+1: n_rows+n_cols+n_low]
        s_u[iupp] += α_dual_final * Δ[n_rows+n_cols+n_low+1: end]
        
        n_Δx = α_pri * norm(Δ[1:n_cols])
        μ = compute_μ(x[ilow], x[iupp], 
                      s_l[ilow], s_u[iupp], 
                      lvar[ilow], uvar[iupp],
                      n_low, n_upp)
        Qx = Q * x
        xTQx_2 = x' * Qx / 2
        cTx = c' * x
        pri_obj = xTQx_2 + cTx + c0
        dual_obj = b' * λ - xTQx_2 + s_l[ilow]'*lvar[ilow] - s_u[iupp]'*uvar[iupp] +c0

        rb = A * x - b
        rc = -Qx + A' * λ + s_l - s_u - c
        
        # update stopping criterion values:
        
        pdd = abs(pri_obj - dual_obj ) / (1 + abs(pri_obj)) 
        max_rc, max_rb = norm(rc, Inf), norm(rb, Inf)
        optimal = pdd < ϵ_pdd && max_rb < ϵ_rb && max_rc < ϵ_rc
        small_Δx, small_μ = n_Δx < tol_Δx, μ < ϵ_μ
        k += 1
        
                    ### case ρ=δ and ρ /= 10 each iter
#         if ρ >= 1e-7
#             for j in n_cols+1:n_rows+n_cols
#                 J_augm[j,j] /= 10.
#             end
#             ρ /= 10.
#         end
        
        if display == true
            @info log_row(Any[k, pri_obj, pdd, max_rb, max_rc, n_Δx, μ])
        end
        
        
    end
    
    elapsed_time = time() - start_time
    
    if k>= max_iter
        status = :max_iter
    else
        status = :acceptable
    end
    
    stats = GenericExecutionStats(status, QM, solution = x,
                                  objective = pri_obj , 
                                  dual_feas = max_rc, 
                                  primal_feas = max_rb,
                                  multipliers = λ,
                                  multipliers_L = s_l,
                                  multipliers_U = s_u,
                                  iter = k, elapsed_time=elapsed_time)
    return stats
end



mehrotraPCQuadBounds (generic function with 1 method)

### bibliothèque QuadraticModels

optimize $c_0 + c'x + \frac{1}{2} x'Hx ~~~~$ s.t. $~~L \le Ax \le U$ and $l \le x \le u$ 

Ici L = lcon, U = ucon, l = lvar, u = uvar

In [194]:
# probleme1
Q = [6 2 1
    2 5 2
    1 2 4]
c = [-8; -3; -3]
c0 = 0.
A = [1 0 1
    0 1 1]
b = [0; 3]
lvar = [0;0;0]
uvar = [Inf; Inf; Inf]
lcon = b
ucon = b

x01 = [1.; 2.; 3.];


In [195]:
QM = QuadraticModel(c, Q, A=A, lcon=lcon, ucon=ucon, lvar=lvar, uvar=uvar, x0=x01, c0=c0, name="QM1")
SM = SlackModel(QM)
typeof(SM)

QuadraticModel

In [196]:
stats_mpc1 =  mehrotraPCQuadBounds(SM)
println(stats_mpc1)

┌ Info:   Iter    primal       pdd   rb cond   rc cond      ‖Δx‖         μ  
└ @ Main In[193]:65


Generic Execution stats
  status: solved to within acceptable tolerances
  objective value: 13.500000000002306
  primal feasibility: 1.9984014443252818e-13
  dual feasibility: 4.248708052045913e-10
  solution: [1.5305844037475238e-14  3.0000000000001985  1.1193513346675583e-15]
  multipliers: [-15.2699535510931  12.000000000123299]
  multipliers_L: [7.269953551093189  1.8145479218559824e-14  6.269953551395089]
  multipliers_U: [0.0  0.0  0.0]
  iterations: 6
  elapsed time: 0.6560001373291016


┌ Info:      0   3.1e+01   1.7e+00   2.5e+00   6.5e+00   0.0e+00   6.4e+00
└ @ Main In[193]:70
┌ Info:      1   1.4e+01   3.3e-02   3.9e-01   1.4e+00   1.7e+00   8.4e-01
└ @ Main In[193]:164
┌ Info:      2   1.4e+01   1.4e-02   1.6e-02   1.1e-01   3.9e-01   5.6e-02
└ @ Main In[193]:164
┌ Info:      3   1.4e+01   6.3e-03   1.6e-05   4.6e-02   2.2e-02   5.8e-05
└ @ Main In[193]:164
┌ Info:      4   1.4e+01   1.3e-05   4.6e-08   9.1e-05   2.2e-05   5.8e-08
└ @ Main In[193]:164
┌ Info:      5   1.4e+01   1.9e-08   1.1e-10   2.0e-07   4.8e-08   5.8e-11
└ @ Main In[193]:164
┌ Info:      6   1.4e+01   2.5e-11   2.0e-13   4.2e-10   1.1e-10   5.8e-14
└ @ Main In[193]:164


In [197]:
#pb verif

QM_verif = QuadraticModel(c*(-1), Q, A=A*(-1), lcon=lcon, ucon=ucon, lvar=-uvar, uvar=lvar, x0=x01, c0=c0, name="QM1")
SM_verif = SlackModel(QM_verif)
stats_mpc_verif =  mehrotraPCQuadBounds(SM_verif);
println(stats_mpc_verif)

┌ Info:   Iter    primal       pdd   rb cond   rc cond      ‖Δx‖         μ  
└ @ Main In[193]:65


Generic Execution stats
  status: solved to within acceptable tolerances
  objective value: 13.500000000002306
  primal feasibility: 1.9984014443252818e-13
  dual feasibility: 4.248708052045913e-10
  solution: [-1.5305844037475238e-14  -3.0000000000001985  -1.1193513346675583e-15]
  multipliers: [-15.2699535510931  12.000000000123299]
  multipliers_L: [0.0  0.0  0.0]
  multipliers_U: [7.269953551093189  1.8145479218559824e-14  6.269953551395089]
  iterations: 6
  elapsed time: 0.6499998569488525


┌ Info:      0   3.1e+01   1.7e+00   2.5e+00   6.5e+00   0.0e+00   6.4e+00
└ @ Main In[193]:70
┌ Info:      1   1.4e+01   3.3e-02   3.9e-01   1.4e+00   1.7e+00   8.4e-01
└ @ Main In[193]:164
┌ Info:      2   1.4e+01   1.4e-02   1.6e-02   1.1e-01   3.9e-01   5.6e-02
└ @ Main In[193]:164
┌ Info:      3   1.4e+01   6.3e-03   1.6e-05   4.6e-02   2.2e-02   5.8e-05
└ @ Main In[193]:164
┌ Info:      4   1.4e+01   1.3e-05   4.6e-08   9.1e-05   2.2e-05   5.8e-08
└ @ Main In[193]:164
┌ Info:      5   1.4e+01   1.9e-08   1.1e-10   2.0e-07   4.8e-08   5.8e-11
└ @ Main In[193]:164
┌ Info:      6   1.4e+01   2.5e-11   2.0e-13   4.2e-10   1.1e-10   5.8e-14
└ @ Main In[193]:164


In [198]:
### pb verif 2
c2 = [1; 0; 0]
A2 = Matrix([1, 1, 1]')
b2 = [1];
QM_verif2 = QuadraticModel(c2, zeros(3,3), A=A2, lcon=b2, ucon=b2, lvar=[0.;0;0], 
                uvar=[Inf; Inf; Inf], x0=x01, c0=c0, name="QMverfi2")
SM_verif2 = SlackModel(QM_verif2)
stats_mpc_verif2 =  mehrotraPCQuadBounds(SM_verif2);
println(stats_mpc_verif2)

┌ Info:   Iter    primal       pdd   rb cond   rc cond      ‖Δx‖         μ  
└ @ Main In[193]:65
┌ Info:      0   1.6e+06   1.0e+00   2.9e+06   9.7e-01   0.0e+00   1.2e+06
└ @ Main In[193]:70


Generic Execution stats
  status: solved to within acceptable tolerances
  objective value: 5.352156597630519e-10
  primal feasibility: 5.95745675013859e-13
  dual feasibility: 1.3100631690576847e-14
  solution: [5.352156597630519e-10  0.49999999973209425  0.49999999973209425]
  multipliers: [-1.9882315318739792e-10]
  multipliers_L: [1.00000000019881  1.9882969377117875e-10  1.9882969377117875e-10]
  multipliers_U: [0.0  0.0  0.0]
  iterations: 6
  elapsed time: 0.7130000591278076


┌ Info:      1   1.8e+03   1.0e+00   3.1e+03   1.6e-02   1.9e+06   1.3e+03
└ @ Main In[193]:164
┌ Info:      2   1.8e+00   8.7e-01   2.5e+00   1.8e-05   2.0e+03   1.3e+00
└ @ Main In[193]:164
┌ Info:      3   2.4e-01   3.5e-01   4.4e-07   1.6e-08   1.7e+00   1.5e-01
└ @ Main In[193]:164
┌ Info:      4   5.4e-04   7.3e-04   2.0e-07   2.4e-09   2.9e-01   2.4e-04
└ @ Main In[193]:164
┌ Info:      5   5.4e-07   7.3e-07   4.0e-10   7.7e-12   6.5e-04   2.4e-07
└ @ Main In[193]:164
┌ Info:      6   5.4e-10   7.3e-10   6.0e-13   1.3e-14   6.6e-07   2.4e-10
└ @ Main In[193]:164


# Lecture des donnees .SIF

In [199]:

function createQuadraticModel(qpdata; name="qp_pb")
    return QuadraticModel(qpdata.c, qpdata.qrows, qpdata.qcols, qpdata.qvals,
            Arows=qpdata.arows, Acols=qpdata.acols, Avals=qpdata.avals, 
            lcon=qpdata.lcon, ucon=qpdata.ucon, lvar=qpdata.lvar, uvar=qpdata.uvar,
            c0=qpdata.c0, name=name)
end

function displayQuadraticModel(QM)
    #println("A = ", Matrix(jac(QM, QM.meta.x0)))
    #println("Q = ", Matrix(hess(QM, QM.meta.x0)))
    println("lvar = ", QM.meta.lvar)
    println("uvar = ", QM.meta.uvar)
    println("x0 = ", QM.meta.x0)
    #println("lcon = ", QM.meta.lcon)
    #println("ucon = ", QM.meta.ucon)
end

displayQuadraticModel (generic function with 1 method)

# Benchmark

In [200]:
path_pb = "C:\\Users\\Geoffroy Leconte\\Documents\\cours\\TFE\\code\\problemes_netlib"
afiro = string(path_pb, "\\AFIRO.SIF")

"C:\\Users\\Geoffroy Leconte\\Documents\\cours\\TFE\\code\\problemes_netlib\\AFIRO.SIF"

In [201]:
qpdata2 = readqps(afiro)  # obj   -4.6475314286E+02
qpdata2.contypes
QM2 = createQuadraticModel(qpdata2)
SM2 = SlackModel(QM2);
SM2.meta

┌ Info: Using 'AFIRO' as NAME (l. 5)
└ @ QPSReader C:\Users\Geoffroy Leconte\.julia\packages\QPSReader\usDQr\src\readqps.jl:751
┌ Info: Using 'COST' as objective (l. 47)
└ @ QPSReader C:\Users\Geoffroy Leconte\.julia\packages\QPSReader\usDQr\src\readqps.jl:350
┌ Info: Using 'B' as RHS (l. 96)
└ @ QPSReader C:\Users\Geoffroy Leconte\.julia\packages\QPSReader\usDQr\src\readqps.jl:470


  Problem name: qp_pb-slack
   All variables: ████████████████████ 51     All constraints: ████████████████████ 27    
            free: ⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅ 0                 free: ⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅ 0     
           lower: █████████████⋅⋅⋅⋅⋅⋅⋅ 32               lower: ⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅ 0     
           upper: ████████⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅ 19               upper: ⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅ 0     
         low/upp: ⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅ 0              low/upp: ⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅ 0     
           fixed: ⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅ 0                fixed: ████████████████████ 27    
          infeas: ⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅ 0               infeas: ⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅ 0     
            nnzh: (100.00% sparsity)   0               linear: ████████████████████ 27    
                                                    nonlinear: ⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅ 0     
                                                         nnzj: ( 92.59% sparsity)   102   



In [202]:
stats_mpc2 =  mehrotraPCQuadBounds(SM2)
println(stats_mpc2)

Generic Execution stats
  status: solved to within acceptable tolerances
  objective value: -464.7531428536097
  primal feasibility: 2.2737367544323206e-13
  dual feasibility: 7.227551890309769e-14
  solution: [79.99999999972938  25.49999999945909  54.50000000027025  84.7999999997133 ⋯ 103.82003141767206]
  multipliers: [-0.6285714285872157  5.154372543820054e-11  -0.34477142853494114  -0.2285714285766529 ⋯ -4.74395588429957e-13]
  multipliers_L: [1.2688290366012446e-12  1.057539304624286e-11  1.635170638877493e-12  1.1974268720829814e-12 ⋯ 0.0]
  multipliers_U: [0.0  0.0  0.0  0.0 ⋯ 4.665982627529523e-13]
  iterations: 11
  elapsed time: 0.003999948501586914


┌ Info:   Iter    primal       pdd   rb cond   rc cond      ‖Δx‖         μ  
└ @ Main In[193]:65
┌ Info:      0   1.0e+08   1.0e+00   7.8e+07   4.2e+00   0.0e+00   1.8e+07
└ @ Main In[193]:70
┌ Info:      1   1.2e+07   1.0e+00   1.5e+07   1.2e-01   2.5e+07   3.5e+06
└ @ Main In[193]:164
┌ Info:      2   1.7e+04   1.7e+00   1.8e+04   2.9e-02   7.7e+06   4.5e+03
└ @ Main In[193]:164
┌ Info:      3   1.7e+03   6.3e+00   2.0e+03   3.4e-05   8.4e+03   5.9e+02
└ @ Main In[193]:164
┌ Info:      4  -1.2e+02   3.6e+01   2.9e+01   1.3e-05   1.1e+03   9.3e+01
└ @ Main In[193]:164
┌ Info:      5  -2.0e+02   4.2e+00   5.3e-06   2.3e-06   6.9e+01   1.7e+01
└ @ Main In[193]:164
┌ Info:      6  -4.1e+02   4.4e-01   1.5e-06   2.5e-06   4.1e+02   3.6e+00
└ @ Main In[193]:164
┌ Info:      7  -4.6e+02   7.5e-02   6.1e-07   9.8e-07   9.2e+01   6.7e-01
└ @ Main In[193]:164
┌ Info:      8  -4.6e+02   7.4e-03   2.1e-07   8.8e-08   1.8e+01   6.7e-02
└ @ Main In[193]:164
┌ Info:      9  -4.6e+02   1.4e-05   5.3

In [203]:
@benchmark mehrotraPCQuadBounds(SM2,  display=false)

BenchmarkTools.Trial: 
  memory estimate:  1.55 MiB
  allocs estimate:  10181
  --------------
  minimum time:     1.508 ms (0.00% GC)
  median time:      1.556 ms (0.00% GC)
  mean time:        1.800 ms (4.77% GC)
  maximum time:     11.772 ms (49.47% GC)
  --------------
  samples:          2771
  evals/sample:     1

In [204]:
# problem 3   kb2    obj  -1.7499001299E+03
kb2 = string(path_pb, "\\KB2.SIF")
qpdata3 = readqps(kb2)
QM3 = createQuadraticModel(qpdata3)
SM3 = SlackModel(QM3)
SM3.meta

┌ Info: Using 'KB2' as NAME (l. 5)
└ @ QPSReader C:\Users\Geoffroy Leconte\.julia\packages\QPSReader\usDQr\src\readqps.jl:751
┌ Info: Using 'FAT7..J.' as objective (l. 20)
└ @ QPSReader C:\Users\Geoffroy Leconte\.julia\packages\QPSReader\usDQr\src\readqps.jl:350
┌ Info: Using '77BOUND' as BOUNDS (l. 227)
└ @ QPSReader C:\Users\Geoffroy Leconte\.julia\packages\QPSReader\usDQr\src\readqps.jl:622


  Problem name: qp_pb-slack
   All variables: ████████████████████ 68     All constraints: ████████████████████ 43    
            free: ⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅ 0                 free: ⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅ 0     
           lower: ██████████████⋅⋅⋅⋅⋅⋅ 47               lower: ⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅ 0     
           upper: ████⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅ 12               upper: ⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅ 0     
         low/upp: ███⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅ 9              low/upp: ⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅ 0     
           fixed: ⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅ 0                fixed: ████████████████████ 43    
          infeas: ⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅ 0               infeas: ⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅ 0     
            nnzh: (100.00% sparsity)   0               linear: ████████████████████ 43    
                                                    nonlinear: ⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅ 0     
                                                         nnzj: ( 89.30% sparsity)   313   



In [205]:
stats_mpc3 =  mehrotraPCQuadBounds(SM3);
println(stats_mpc3)

Generic Execution stats
  status: solved to within acceptable tolerances
  objective value: -1749.900128971516
  primal feasibility: 2.2716051262250403e-10
  dual feasibility: 3.4508586997996273e-9
  solution: [0.8118292158464473  1.0578894239843056e-7  4.950583786096597e-7  2.1425963651756203e-7 ⋯ -783.4592932499002]
  multipliers: [17.2692083564685  17.214101749509137  16.942045562868106  16.659838299711776 ⋯ -1.7029914497648327e-9]
  multipliers_L: [8.871707035068856e-9  0.06381185576540346  0.04896143735044444  0.10208276588002066 ⋯ 0.0]
  multipliers_U: [0.0  3.829548487213249e-10  0.0  0.0 ⋯ 5.78860055896958e-12]
  iterations: 16
  elapsed time: 0.01699995994567871


┌ Info:   Iter    primal       pdd   rb cond   rc cond      ‖Δx‖         μ  
└ @ Main In[193]:65
┌ Info:      0   1.4e+06   1.0e+00   5.6e+09   1.4e+01   0.0e+00   7.2e+07
└ @ Main In[193]:70
┌ Info:      1   1.4e+06   8.9e+02   5.6e+09   3.6e-01   1.9e+04   1.2e+08
└ @ Main In[193]:164
┌ Info:      2   4.4e+03   1.1e+03   1.5e+07   1.4e-01   7.4e+07   3.5e+05
└ @ Main In[193]:164
┌ Info:      3  -2.4e+02   3.0e+03   1.6e+06   2.0e-02   2.5e+05   4.0e+04
└ @ Main In[193]:164
┌ Info:      4  -1.2e+03   3.9e+02   2.8e+05   5.7e-03   2.5e+04   1.2e+04
└ @ Main In[193]:164
┌ Info:      5  -1.3e+03   1.4e+02   7.9e+04   2.0e-03   5.9e+03   4.0e+03
└ @ Main In[193]:164
┌ Info:      6  -1.4e+03   1.8e+01   2.4e+03   1.1e-04   4.0e+03   3.6e+02
└ @ Main In[193]:164
┌ Info:      7  -1.4e+03   4.7e+00   9.8e+02   2.7e-05   2.7e+02   9.7e+01
└ @ Main In[193]:164
┌ Info:      8  -1.6e+03   1.2e+00   6.4e-04   8.6e-06   1.5e+03   2.5e+01
└ @ Main In[193]:164
┌ Info:      9  -1.7e+03   1.1e-01   2.4

In [206]:
@benchmark mehrotraPCQuadBounds(SM3, display = false)

BenchmarkTools.Trial: 
  memory estimate:  3.64 MiB
  allocs estimate:  17826
  --------------
  minimum time:     3.178 ms (0.00% GC)
  median time:      3.289 ms (0.00% GC)
  mean time:        3.652 ms (4.34% GC)
  maximum time:     9.484 ms (39.20% GC)
  --------------
  samples:          1368
  evals/sample:     1

In [214]:
# problem 4 SC50A  obj  -6.4575077059E+01
SC50A = string(path_pb, "\\25FV47.SIF")
qpdata4 = readqps(SC50A)
QM4 = createQuadraticModel(qpdata4)
SM4 = SlackModel(QM4)
SM4.meta

┌ Info: Using '25FV47' as NAME (l. 5)
└ @ QPSReader C:\Users\Geoffroy Leconte\.julia\packages\QPSReader\usDQr\src\readqps.jl:751
┌ Info: Using 'R0000' as objective (l. 21)
└ @ QPSReader C:\Users\Geoffroy Leconte\.julia\packages\QPSReader\usDQr\src\readqps.jl:350
┌ Info: Using '.00001' as RHS (l. 6793)
└ @ QPSReader C:\Users\Geoffroy Leconte\.julia\packages\QPSReader\usDQr\src\readqps.jl:470


  Problem name: qp_pb-slack
   All variables: ████████████████████ 1876   All constraints: ████████████████████ 821   
            free: ⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅ 0                 free: ⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅ 0     
           lower: █████████████████⋅⋅⋅ 1571             lower: ⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅ 0     
           upper: ████⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅ 305              upper: ⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅ 0     
         low/upp: ⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅ 0              low/upp: ⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅ 0     
           fixed: ⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅ 0                fixed: ████████████████████ 821   
          infeas: ⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅ 0               infeas: ⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅ 0     
            nnzh: (100.00% sparsity)   0               linear: ████████████████████ 821   
                                                    nonlinear: ⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅ 0     
                                                         nnzj: ( 99.30% sparsity)   10705 



In [215]:
stats_mpc4 =  mehrotraPCQuadBounds(SM4);
println(stats_mpc4)

┌ Info:   Iter    primal       pdd   rb cond   rc cond      ‖Δx‖         μ  
└ @ Main In[193]:65
┌ Info:      0   4.5e+10   1.0e+00   4.1e+10   5.1e+01   0.0e+00   2.2e+09
└ @ Main In[193]:70
┌ Info:      1   4.0e+10   1.0e+00   3.5e+10   3.5e+01   4.9e+08   2.2e+09
└ @ Main In[193]:164
┌ Info:      2   1.9e+10   1.0e+00   1.5e+10   1.3e+01   1.1e+09   1.0e+09
└ @ Main In[193]:164
┌ Info:      3   3.6e+09   1.0e+00   2.6e+09   1.4e+00   7.1e+08   1.8e+08
└ @ Main In[193]:164
┌ Info:      4   9.2e+08   1.0e+00   5.1e+08   1.2e+00   4.3e+08   3.8e+07
└ @ Main In[193]:164
┌ Info:      5   2.4e+08   1.0e+00   7.2e+07   1.1e+00   2.4e+08   5.6e+06
└ @ Main In[193]:164
┌ Info:      6   6.9e+07   1.0e+00   3.1e+06   3.7e-01   9.3e+07   2.5e+05
└ @ Main In[193]:164
┌ Info:      7   4.9e+07   1.1e+00   1.2e+06   2.3e-01   3.5e+07   1.4e+05
└ @ Main In[193]:164
┌ Info:      8   1.5e+07   1.2e+00   1.0e+05   2.5e-01   5.1e+07   1.3e+04
└ @ Main In[193]:164
┌ Info:      9   1.6e+07   1.2e+00   3.0

┌ Info:     91       NaN       NaN       NaN       NaN       NaN       NaN
└ @ Main In[193]:164
┌ Info:     92       NaN       NaN       NaN       NaN       NaN       NaN
└ @ Main In[193]:164
┌ Info:     93       NaN       NaN       NaN       NaN       NaN       NaN
└ @ Main In[193]:164
┌ Info:     94       NaN       NaN       NaN       NaN       NaN       NaN
└ @ Main In[193]:164
┌ Info:     95       NaN       NaN       NaN       NaN       NaN       NaN
└ @ Main In[193]:164
┌ Info:     96       NaN       NaN       NaN       NaN       NaN       NaN
└ @ Main In[193]:164
┌ Info:     97       NaN       NaN       NaN       NaN       NaN       NaN
└ @ Main In[193]:164
┌ Info:     98       NaN       NaN       NaN       NaN       NaN       NaN
└ @ Main In[193]:164
┌ Info:     99       NaN       NaN       NaN       NaN       NaN       NaN
└ @ Main In[193]:164
┌ Info:    100       NaN       NaN       NaN       NaN       NaN       NaN
└ @ Main In[193]:164
┌ Info:    101       NaN       NaN      

┌ Info:    184       NaN       NaN       NaN       NaN       NaN       NaN
└ @ Main In[193]:164
┌ Info:    185       NaN       NaN       NaN       NaN       NaN       NaN
└ @ Main In[193]:164
┌ Info:    186       NaN       NaN       NaN       NaN       NaN       NaN
└ @ Main In[193]:164
┌ Info:    187       NaN       NaN       NaN       NaN       NaN       NaN
└ @ Main In[193]:164
┌ Info:    188       NaN       NaN       NaN       NaN       NaN       NaN
└ @ Main In[193]:164
┌ Info:    189       NaN       NaN       NaN       NaN       NaN       NaN
└ @ Main In[193]:164
┌ Info:    190       NaN       NaN       NaN       NaN       NaN       NaN
└ @ Main In[193]:164
┌ Info:    191       NaN       NaN       NaN       NaN       NaN       NaN
└ @ Main In[193]:164
┌ Info:    192       NaN       NaN       NaN       NaN       NaN       NaN
└ @ Main In[193]:164
┌ Info:    193       NaN       NaN       NaN       NaN       NaN       NaN
└ @ Main In[193]:164
┌ Info:    194       NaN       NaN      

┌ Info:    271       NaN       NaN       NaN       NaN       NaN       NaN
└ @ Main In[193]:164
┌ Info:    272       NaN       NaN       NaN       NaN       NaN       NaN
└ @ Main In[193]:164
┌ Info:    273       NaN       NaN       NaN       NaN       NaN       NaN
└ @ Main In[193]:164
┌ Info:    274       NaN       NaN       NaN       NaN       NaN       NaN
└ @ Main In[193]:164
┌ Info:    275       NaN       NaN       NaN       NaN       NaN       NaN
└ @ Main In[193]:164
┌ Info:    276       NaN       NaN       NaN       NaN       NaN       NaN
└ @ Main In[193]:164
┌ Info:    277       NaN       NaN       NaN       NaN       NaN       NaN
└ @ Main In[193]:164
┌ Info:    278       NaN       NaN       NaN       NaN       NaN       NaN
└ @ Main In[193]:164
┌ Info:    279       NaN       NaN       NaN       NaN       NaN       NaN
└ @ Main In[193]:164
┌ Info:    280       NaN       NaN       NaN       NaN       NaN       NaN
└ @ Main In[193]:164
┌ Info:    281       NaN       NaN      

Generic Execution stats
  status: maximum iteration
  objective value: NaN
  primal feasibility: NaN
  dual feasibility: NaN
  solution: [NaN  NaN  NaN  NaN ⋯ NaN]
  multipliers: [0.0  NaN  NaN  NaN ⋯ NaN]
  multipliers_L: [NaN  NaN  NaN  NaN ⋯ 0.0]
  multipliers_U: [0.0  0.0  0.0  0.0 ⋯ NaN]
  iterations: 300
  elapsed time: 3.7019999027252197


┌ Info:    295       NaN       NaN       NaN       NaN       NaN       NaN
└ @ Main In[193]:164
┌ Info:    296       NaN       NaN       NaN       NaN       NaN       NaN
└ @ Main In[193]:164
┌ Info:    297       NaN       NaN       NaN       NaN       NaN       NaN
└ @ Main In[193]:164
┌ Info:    298       NaN       NaN       NaN       NaN       NaN       NaN
└ @ Main In[193]:164
┌ Info:    299       NaN       NaN       NaN       NaN       NaN       NaN
└ @ Main In[193]:164
┌ Info:    300       NaN       NaN       NaN       NaN       NaN       NaN
└ @ Main In[193]:164


# Tests sur tous les problèmes

In [216]:
df_test = DataFrame(id = Int[],
         name = String[],
         status = Symbol[],
         objective = Float64[],
         time = Float64[],
         iter = Int[],
         primal_feas = Float64[],
         dual_feas = Float64[])



i = 0
i_stop = 4   #number of problems to solve
for file_name in readdir(path_pb)
    if file_name[end-3:end] == ".SIF" && !(file_name in ["80BAU3B.SIF" ; "BORE3D.SIF"; 
                                                        "CAPRI.SIF"; "CZPROB.SIF"; 
                                                        "ETAMACRO.SIF"; "FINNIS.SIF";
                                                        "FORPLAN.SIF"; "GREENBEA.SIF";
                                                        "GREENBEB.SIF"; "MAROS.SIF";
                                                        "NESM.SIF"; "PEROLD.SIF";
                                                         "PILOT-JA.SIF"; "PILOT-WE.SIF";
                                                         "PILOT.SIF"; "PILOT4.SIF";
                                                         "PILOT87.SIF"; "PILOTNOV.SIF";
                                                          "RECIPELP.SIF"; "SHELL.SIF";
                                                         "SIERRA.SIF"; "STAIR.SIF";
                                                         "STANDATA.SIF"; "STANDGUB.SIF";
                                                        "STANDMPS.SIF"; "TUFF.SIF";
                                                        "VTP-BASE.SIF"])   &&# problems with fixed variables
                                        !(file_name in ["25FV47.SIF"; "BNL1.SIF";
                                                        "BRANDY.SIF"; "CRE-A.SIF"]) # singular error
                            
        
        i += 1
        println(file_name)
        pb_i = string(path_pb, "\\", file_name)
        if file_name in ["BLEND.SIF"; "DFL001.SIF"; "FORPLAN.SIF"; "GFRD-PNC.SIF"; "SIERRA.SIF"]
            qpdata_i = readqps(pb_i, mpsformat=:fixed) 
        else
            qpdata_i = readqps(pb_i) 
        end

        SM_i = SlackModel(createQuadraticModel(qpdata_i))

        stats_mpc_i =  mehrotraPCQuadBounds(SM_i, display=false)
        
        # 
        push!(df_test, Dict(:id => i,
                             :name => file_name ,
                             :status => stats_mpc_i.status,
                             :objective => stats_mpc_i.objective,
                             :time => stats_mpc_i.elapsed_time,
                             :iter => stats_mpc_i.iter,
                             :primal_feas => stats_mpc_i.primal_feas,
                             :dual_feas => stats_mpc_i.dual_feas))
    end
    
    if i == i_stop
        break
        break
    end
end

markdown_table(stdout, df_test)

ADLITTLE.SIF
AFIRO.SIF
AGG.SIF
AGG2.SIF


┌ Info: Using 'ADLITTLE' as NAME (l. 5)
└ @ QPSReader C:\Users\Geoffroy Leconte\.julia\packages\QPSReader\usDQr\src\readqps.jl:751
┌ Info: Using '.Z....' as objective (l. 20)
└ @ QPSReader C:\Users\Geoffroy Leconte\.julia\packages\QPSReader\usDQr\src\readqps.jl:350
┌ Info: Using 'ZZZZ0001' as RHS (l. 333)
└ @ QPSReader C:\Users\Geoffroy Leconte\.julia\packages\QPSReader\usDQr\src\readqps.jl:470
┌ Info: Using 'AFIRO' as NAME (l. 5)
└ @ QPSReader C:\Users\Geoffroy Leconte\.julia\packages\QPSReader\usDQr\src\readqps.jl:751
┌ Info: Using 'COST' as objective (l. 47)
└ @ QPSReader C:\Users\Geoffroy Leconte\.julia\packages\QPSReader\usDQr\src\readqps.jl:350
┌ Info: Using 'B' as RHS (l. 96)
└ @ QPSReader C:\Users\Geoffroy Leconte\.julia\packages\QPSReader\usDQr\src\readqps.jl:470
┌ Info: Using 'AGG' as NAME (l. 5)
└ @ QPSReader C:\Users\Geoffroy Leconte\.julia\packages\QPSReader\usDQr\src\readqps.jl:751
┌ Info: Using 'OBJECTIV' as objective (l. 509)
└ @ QPSReader C:\Users\Geoffroy Leconte\.jul

|    id |         name |     status | objective |     time |  iter | primal_fe ⋯
|-------|--------------|------------|-----------|----------|-------|---------- ⋯
|     1 | ADLITTLE.SIF | acceptable |   2.3e+05 |  5.0e-03 |    14 |     1.6e- ⋯
|     2 |    AFIRO.SIF | acceptable |  -4.6e+02 |  2.0e-03 |    11 |     2.3e- ⋯
|     3 |      AGG.SIF | acceptable |  -3.6e+07 |  4.2e-02 |    23 |     5.9e- ⋯
|     4 |     AGG2.SIF | acceptable |  -2.0e+07 |  9.3e-02 |    22 |     1.2e- ⋯


┌ Info: Using 'RHS' as RHS (l. 2878)
└ @ QPSReader C:\Users\Geoffroy Leconte\.julia\packages\QPSReader\usDQr\src\readqps.jl:470


In [217]:
T = typeof(SM2.meta)
for (name, typ) in zip(fieldnames(T), T.types)
    println("type of the fieldname $name is $typ")
end

type of the fieldname nvar is Int64
type of the fieldname x0 is Array{T,1} where T
type of the fieldname lvar is Array{T,1} where T
type of the fieldname uvar is Array{T,1} where T
type of the fieldname ifix is Array{Int64,1}
type of the fieldname ilow is Array{Int64,1}
type of the fieldname iupp is Array{Int64,1}
type of the fieldname irng is Array{Int64,1}
type of the fieldname ifree is Array{Int64,1}
type of the fieldname iinf is Array{Int64,1}
type of the fieldname nbv is Int64
type of the fieldname niv is Int64
type of the fieldname nlvb is Int64
type of the fieldname nlvo is Int64
type of the fieldname nlvc is Int64
type of the fieldname nlvbi is Int64
type of the fieldname nlvci is Int64
type of the fieldname nlvoi is Int64
type of the fieldname nwv is Int64
type of the fieldname ncon is Int64
type of the fieldname y0 is Array{T,1} where T
type of the fieldname lcon is Array{T,1} where T
type of the fieldname ucon is Array{T,1} where T
type of the fieldname jfix is Array{Int64,1

In [218]:
# problems: AGG.SIF
qpdata5 = readqps(string(path_pb, "\\AGG2.SIF"))
QM5 = createQuadraticModel(qpdata5)
SM5 = SlackModel(QM5);

┌ Info: Using 'AGG2' as NAME (l. 5)
└ @ QPSReader C:\Users\Geoffroy Leconte\.julia\packages\QPSReader\usDQr\src\readqps.jl:751
┌ Info: Using 'OBJECTIV' as objective (l. 536)
└ @ QPSReader C:\Users\Geoffroy Leconte\.julia\packages\QPSReader\usDQr\src\readqps.jl:350
┌ Info: Using 'RHS' as RHS (l. 2878)
└ @ QPSReader C:\Users\Geoffroy Leconte\.julia\packages\QPSReader\usDQr\src\readqps.jl:470


In [219]:
stats_mpc_5 =  mehrotraPCQuadBounds(SM5, max_iter=150)
println(stats_mpc_5)

┌ Info:   Iter    primal       pdd   rb cond   rc cond      ‖Δx‖         μ  
└ @ Main In[193]:65
┌ Info:      0   8.2e+11   1.0e+00   3.1e+10   7.5e+01   0.0e+00   3.9e+09
└ @ Main In[193]:70
┌ Info:      1   4.7e+11   1.0e+00   1.2e+10   5.2e+01   1.2e+09   1.8e+09
└ @ Main In[193]:164
┌ Info:      2   8.8e+10   1.0e+00   9.2e+08   1.8e+00   9.1e+08   1.8e+08
└ @ Main In[193]:164
┌ Info:      3   1.7e+09   1.3e+00   1.8e+07   2.0e-01   1.4e+08   4.1e+06
└ @ Main In[193]:164
┌ Info:      4   9.5e+08   1.4e+00   9.6e+06   6.7e-02   1.3e+06   2.6e+06
└ @ Main In[193]:164
┌ Info:      5   3.2e+08   1.5e+00   3.4e+06   9.9e-03   1.1e+06   9.1e+05
└ @ Main In[193]:164
┌ Info:      6   8.3e+07   2.5e+00   9.5e+05   2.0e-03   4.6e+05   3.5e+05
└ @ Main In[193]:164
┌ Info:      7  -2.3e+06   2.2e+01   1.8e+05   2.1e-03   3.0e+05   7.6e+04
└ @ Main In[193]:164
┌ Info:      8  -6.2e+06   7.1e+00   1.5e+05   1.7e-03   1.0e+05   6.6e+04
└ @ Main In[193]:164
┌ Info:      9  -1.1e+07   3.1e+00   1.1

Generic Execution stats
  status: solved to within acceptable tolerances
  objective value: -2.0239252353329867e7
  primal feasibility: 1.2095687224757209e-8
  dual feasibility: 2.857987749252061e-7
  solution: [15031.233073233205  0.00023919352166962282  0.00025333876799812916  0.0002636789647937343 ⋯ 547999.999995669]
  multipliers: [-9.152874420479009e-10  -1.7714841784741088e-8  -3.011541488292951e-9  -4.911136417811601e-9 ⋯ 69.7812195369353]
  multipliers_L: [1.2337761228446633e-9  0.053999997705932314  0.05099999661705303  0.04899999629351781 ⋯ 0.0]
  multipliers_U: [0.0  0.0  0.0  0.0 ⋯ 2.9849999952450372]
  iterations: 22
  elapsed time: 0.14300012588500977


┌ Info:     17  -2.0e+07   3.9e-03   1.3e+01   3.0e-04   8.2e+04   1.1e+02
└ @ Main In[193]:164
┌ Info:     18  -2.0e+07   6.0e-04   1.5e+00   2.2e-04   4.1e+04   1.6e+01
└ @ Main In[193]:164
┌ Info:     19  -2.0e+07   6.1e-05   3.8e-02   4.8e-05   9.1e+03   1.7e+00
└ @ Main In[193]:164
┌ Info:     20  -2.0e+07   9.1e-06   6.2e-04   7.0e-06   1.1e+03   2.5e-01
└ @ Main In[193]:164
┌ Info:     21  -2.0e+07   4.6e-07   1.2e-05   2.0e-06   4.5e+02   1.3e-02
└ @ Main In[193]:164
┌ Info:     22  -2.0e+07   6.3e-10   1.2e-08   2.9e-07   1.3e+01   1.7e-05
└ @ Main In[193]:164


In [220]:
SM5.meta


  Problem name: qp_pb-slack
   All variables: ████████████████████ 758    All constraints: ████████████████████ 516   
            free: ⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅ 0                 free: ⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅ 0     
           lower: ████████⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅ 302              lower: ⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅ 0     
           upper: █████████████⋅⋅⋅⋅⋅⋅⋅ 456              upper: ⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅ 0     
         low/upp: ⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅ 0              low/upp: ⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅ 0     
           fixed: ⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅ 0                fixed: ████████████████████ 516   
          infeas: ⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅ 0               infeas: ⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅ 0     
            nnzh: (100.00% sparsity)   0               linear: ████████████████████ 516   
                                                    nonlinear: ⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅ 0     
                                                         nnzj: ( 98.79% sparsity)   4740  



In [221]:
SM2.meta.iupp

19-element Array{Int64,1}:
 33
 34
 35
 36
 37
 38
 39
 40
 41
 42
 43
 44
 45
 46
 47
 48
 49
 50
 51

In [222]:
problems = []
i_max = 5
i = 1
for file_name in readdir(path_pb)
    if file_name[end-3:end] == ".SIF" && !(file_name in ["80BAU3B.SIF" ; "BORE3D.SIF"; 
                                                        "CAPRI.SIF"; "CZPROB.SIF"; 
                                                        "ETAMACRO.SIF"; "FINNIS.SIF";
                                                        "FORPLAN.SIF"; "GREENBEA.SIF";
                                                        "GREENBEB.SIF"; "MAROS.SIF";
                                                        "NESM.SIF"; "PEROLD.SIF";
                                                         "PILOT-JA.SIF"; "PILOT-WE.SIF";
                                                         "PILOT.SIF"; "PILOT4.SIF";
                                                         "PILOT87.SIF"; "PILOTNOV.SIF";
                                                          "RECIPELP.SIF"; "SHELL.SIF";
                                                         "SIERRA.SIF"; "STAIR.SIF";
                                                         "STANDATA.SIF"; "STANDGUB.SIF";
                                                        "STANDMPS.SIF"; "TUFF.SIF";
                                                        "VTP-BASE.SIF"])   # problems with fixed variables


        println(file_name)
        pb_i = string(path_pb, "\\", file_name)
        if file_name in ["BLEND.SIF"; "DFL001.SIF"; "FORPLAN.SIF"; "GFRD-PNC.SIF"; "SIERRA.SIF"]
            qpdata_i = readqps(pb_i, mpsformat=:fixed) 
        else
            qpdata_i = readqps(pb_i) 
        end
        push!(problems, SlackModel(createQuadraticModel(qpdata_i, name=file_name[1:end-4])))
        
        if i==i_max
            break
        end
        i += 1
    end
end


25FV47.SIF


┌ Info: Using '25FV47' as NAME (l. 5)
└ @ QPSReader C:\Users\Geoffroy Leconte\.julia\packages\QPSReader\usDQr\src\readqps.jl:751


ADLITTLE.SIF
AFIRO.SIF
AGG.SIF
AGG2.SIF


┌ Info: Using 'R0000' as objective (l. 21)
└ @ QPSReader C:\Users\Geoffroy Leconte\.julia\packages\QPSReader\usDQr\src\readqps.jl:350
┌ Info: Using '.00001' as RHS (l. 6793)
└ @ QPSReader C:\Users\Geoffroy Leconte\.julia\packages\QPSReader\usDQr\src\readqps.jl:470
┌ Info: Using 'ADLITTLE' as NAME (l. 5)
└ @ QPSReader C:\Users\Geoffroy Leconte\.julia\packages\QPSReader\usDQr\src\readqps.jl:751
┌ Info: Using '.Z....' as objective (l. 20)
└ @ QPSReader C:\Users\Geoffroy Leconte\.julia\packages\QPSReader\usDQr\src\readqps.jl:350
┌ Info: Using 'ZZZZ0001' as RHS (l. 333)
└ @ QPSReader C:\Users\Geoffroy Leconte\.julia\packages\QPSReader\usDQr\src\readqps.jl:470
┌ Info: Using 'AFIRO' as NAME (l. 5)
└ @ QPSReader C:\Users\Geoffroy Leconte\.julia\packages\QPSReader\usDQr\src\readqps.jl:751
┌ Info: Using 'COST' as objective (l. 47)
└ @ QPSReader C:\Users\Geoffroy Leconte\.julia\packages\QPSReader\usDQr\src\readqps.jl:350
┌ Info: Using 'B' as RHS (l. 96)
└ @ QPSReader C:\Users\Geoffroy Leconte\.ju

In [223]:
problems[2].meta.name

"ADLITTLE-slack"

In [224]:
problems_stats = solve_problems(mehrotraPCQuadBounds, problems, 
                                colstats=[:name, :status, :elapsed_time, :objective, :dual_feas, :primal_feas])

┌ Info:            Name           status      Time      f(x)      Dual    Primal  
└ @ SolverTools C:\Users\Geoffroy Leconte\.julia\packages\SolverTools\whyGk\src\bmark\run_solver.jl:67
┌ Info:    25FV47-slack         max_iter   3.6e+00       NaN       NaN       NaN
└ @ SolverTools C:\Users\Geoffroy Leconte\.julia\packages\SolverTools\whyGk\src\bmark\run_solver.jl:83
┌ Info:  ADLITTLE-slack       acceptable   4.0e-03   2.3e+05   1.8e-08   1.6e-07
└ @ SolverTools C:\Users\Geoffroy Leconte\.julia\packages\SolverTools\whyGk\src\bmark\run_solver.jl:83
┌ Info:     AFIRO-slack       acceptable   1.0e-03  -4.6e+02   7.2e-14   2.3e-13
└ @ SolverTools C:\Users\Geoffroy Leconte\.julia\packages\SolverTools\whyGk\src\bmark\run_solver.jl:83
┌ Info:       AGG-slack       acceptable   4.2e-02  -3.6e+07   2.3e-10   5.9e-09
└ @ SolverTools C:\Users\Geoffroy Leconte\.julia\packages\SolverTools\whyGk\src\bmark\run_solver.jl:83
┌ Info:      AGG2-slack       acceptable   9.2e-02  -2.0e+07   2.9e-07   1.2e-

,id,name,nvar,ncon,nequ,status,objective,elapsed_time
,Int64,String,Int64,Int64,Int64,Symbol,Float64,Float64
1,1,25FV47-slack,1876,821,0,max_iter,NaN,3.584
2,2,ADLITTLE-slack,138,56,0,acceptable,225495.0,0.00400019
3,3,AFIRO-slack,51,27,0,acceptable,-464.753,0.000999928
4,4,AGG-slack,615,488,0,acceptable,-3.59918e7,0.0419998
5,5,AGG2-slack,758,516,0,acceptable,-2.02393e7,0.092


In [225]:
markdown_table(stdout, problems_stats)

|    id |           name |  nvar |  ncon |  nequ |     status | objective | el ⋯
|-------|----------------|-------|-------|-------|------------|-----------|--- ⋯
|     1 |   25FV47-slack |  1876 |   821 |     0 |   max_iter |       NaN |    ⋯
|     2 | ADLITTLE-slack |   138 |    56 |     0 | acceptable |   2.3e+05 |    ⋯
|     3 |    AFIRO-slack |    51 |    27 |     0 | acceptable |  -4.6e+02 |    ⋯
|     4 |      AGG-slack |   615 |   488 |     0 | acceptable |  -3.6e+07 |    ⋯
|     5 |     AGG2-slack |   758 |   516 |     0 | acceptable |  -2.0e+07 |    ⋯
